In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

In [ ]:
# 1. Data Preprocessing
def preprocess_data(df):
    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Extract features from date
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    # Encode vegetable names
    vegetable_encoder = LabelEncoder()
    df['Vegetable_Encoded'] = vegetable_encoder.fit_transform(df['Vegetable'])

    return df, vegetable_encoder

In [ ]:
# 2. Prepare sequences for LSTM
def create_sequences(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
# 3. Build Hybrid Model Class
class HybridPricePredictor:
    def __init__(self, time_steps=5, lstm_units=64):
        self.time_steps = time_steps
        self.lstm_units = lstm_units
        self.scaler = StandardScaler()
        self.rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.lstm_model = None
        self.vegetable_encoder = None

    def build_lstm_model(self, input_shape):
        model = Sequential([
            LSTM(self.lstm_units, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(self.lstm_units//2),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def fit(self, df):
        # Preprocess data
        processed_df, self.vegetable_encoder = preprocess_data(df)

        # Features and target
        features = ['Year', 'Month', 'Day', 'DayOfWeek', 'Vegetable_Encoded']
        X = processed_df[features].values
        y = processed_df['Price'].values

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        # Create sequences for LSTM
        X_seq, y_seq = create_sequences(X_scaled, y, self.time_steps)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

        # Train LSTM
        self.lstm_model = self.build_lstm_model((self.time_steps, X_train.shape[2]))
        history = self.lstm_model.fit(X_train, y_train, epochs=20, batch_size=32,
                                    validation_split=0.1, verbose=1)

        # Get LSTM features
        lstm_features_train = self.lstm_model.predict(X_train)
        lstm_features_test = self.lstm_model.predict(X_test)

        # Combine original features with LSTM features for RF
        X_train_rf = np.hstack((X_train[:, -1, :], lstm_features_train))
        X_test_rf = np.hstack((X_test[:, -1, :], lstm_features_test))

        # Train Random Forest
        self.rf_model.fit(X_train_rf, y_train)

        # Evaluate
        y_pred = self.rf_model.predict(X_test_rf)
        self.evaluate(y_test, y_pred)

        return history

    def predict(self, date, vegetable):
        # Prepare input
        date = pd.to_datetime(date)
        input_df = pd.DataFrame({
            'Date': [date],
            'Vegetable': [vegetable]
        })

        # Preprocess input
        input_df, _ = preprocess_data(input_df)
        features = ['Year', 'Month', 'Day', 'DayOfWeek', 'Vegetable_Encoded']
        X = input_df[features].values

        # Scale features
        X_scaled = self.scaler.transform(X)

        # Create sequence (padding with zeros for single prediction)
        X_seq = np.zeros((1, self.time_steps, X_scaled.shape[1]))
        X_seq[0, -1, :] = X_scaled

        # Get LSTM features
        lstm_features = self.lstm_model.predict(X_seq)

        # Combine features for RF
        X_rf = np.hstack((X_scaled, lstm_features))

        # Predict
        prediction = self.rf_model.predict(X_rf)
        return prediction[0]

    def evaluate(self, y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R2 Score: {r2:.4f}")

        return rmse, mae, r2

In [ ]:
# 4. Main execution
if __name__ == "__main__":
    # Load your data
    df = pd.read_csv('/content/drive/MyDrive/FYPDATA.csv')

    # Initialize and train model
    model = HybridPricePredictor(time_steps=5)
    history = model.fit(df)

    # Example prediction
    date = "2026-08-22"
    vegetable = "Strawberries"
    predicted_price = model.predict(date, vegetable)
    print(f"Predicted price for {vegetable} on {date}: ${predicted_price:.2f}")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


355/355 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.3033 - val_loss: 0.2254
Epoch 2/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2177 - val_loss: 0.2249
Epoch 3/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2142 - val_loss: 0.2240
Epoch 4/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2095 - val_loss: 0.2211
Epoch 5/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.2033 - val_loss: 0.2230
Epoch 6/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.2206 - val_loss: 0.2200
Epoch 7/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.2182 - val_loss: 0.2159
Epoch 8/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.2011 - val_loss: 0.2168
Epoch 9/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.2018 - val_loss: 0.2146
Epoch 10/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.2030 - val_loss: 0.2089
Epoch 11/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.1962 - val_loss: 0.2051
Epoch 12/20
355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: